In [30]:
import asyncio
from bleak import BleakScanner

async def main():
    devices = await BleakScanner.discover(5.0, return_adv=True)
    for d in devices:
        print(d)

await main()
# asyncio.run(main())

38:7D:98:45:F0:26
D0:89:7D:1E:A1:8B


In [31]:
import asyncio
from bleak import BleakScanner, BleakClient

async def main():
    myDevice = ''
    devices = await BleakScanner.discover(5.0, return_adv=True)
    for d in devices:
        if(devices[d][1].local_name == 'Harjoittelua'):
            print("Found it")
            myDevice = d

    address = myDevice
    async with BleakClient(address) as client:
        svcs = client.services
        print("Services:")
        for service in svcs:
            print(service)


await (main())

Found it
Services:
00001801-0000-1000-8000-00805f9b34fb (Handle: 1): Generic Attribute Profile
00001800-0000-1000-8000-00805f9b34fb (Handle: 9): Generic Access Profile
00001523-1212-efde-1523-785feabcd123 (Handle: 16): Nordic LED Button Service


In [ ]:
import asyncio
from bleak import BleakClient

address = "D0:89:7D:1E:A1:8B"
MODEL_NBR_UUID = "00001526-1212-efde-1523-785feabcd123"
"""
00001801-0000-1000-8000-00805f9b34fb (Handle: 1): Generic Attribute Profile
00001800-0000-1000-8000-00805f9b34fb (Handle: 9): Generic Access Profile
00001523-1212-efde-1523-785feabcd123 (Handle: 16): Nordic LED Button Service
"""
async def main(address):
    async with BleakClient(address) as client:
        model_number = await client.read_gatt_char(MODEL_NBR_UUID)
        print("Model Number: {0}".format("".join(map(chr, model_number))))

await (main(address))


TypeError: main() takes 0 positional arguments but 1 was given

In [29]:
import argparse
import asyncio
import logging

from bleak import BleakClient, BleakScanner
from bleak.backends.characteristic import BleakGATTCharacteristic

logger = logging.getLogger(__name__)


def notification_handler(characteristic: BleakGATTCharacteristic, data: bytearray):
    """Simple notification handler which prints the data received."""
    logger.info("%s: %r", characteristic.description, data)


async def main(args: argparse.Namespace):
    logger.info("starting scan...")

    if args.address:
        device = await BleakScanner.find_device_by_address(
            args.address, cb=dict(use_bdaddr=args.macos_use_bdaddr)
        )
        if device is None:
            logger.error("could not find device with address '%s'", args.address)
            return
    else:
        device = await BleakScanner.find_device_by_name(
            args.name, cb=dict(use_bdaddr=args.macos_use_bdaddr)
        )
        if device is None:
            logger.error("could not find device with name '%s'", args.name)
            return

    logger.info("connecting to device...")

    async with BleakClient(device) as client:
        logger.info("Connected")

        await client.start_notify(args.characteristic, notification_handler)
        await asyncio.sleep(5.0)
        await client.stop_notify(args.characteristic)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()

    device_group = parser.add_mutually_exclusive_group(required=True)

    device_group.add_argument(
        "--name",
        metavar="<name>",
        help="the name of the bluetooth device to connect to",
    )
    device_group.add_argument(
        "--address",
        metavar="<address>",
        help="the address of the bluetooth device to connect to",
    )

    parser.add_argument(
        "--macos-use-bdaddr",
        action="store_true",
        help="when true use Bluetooth address instead of UUID on macOS",
    )

    parser.add_argument(
        "characteristic",
        metavar="<notify uuid>",
        help="UUID of a characteristic that supports notifications",
    )

    parser.add_argument(
        "-d",
        "--debug",
        action="store_true",
        help="sets the log level to debug",
    )

    args = parser.parse_args()

    log_level = logging.DEBUG if args.debug else logging.INFO
    logging.basicConfig(
        level=log_level,
        format="%(asctime)-15s %(name)-8s %(levelname)s: %(message)s",
    )

    await (main(args))

usage: ipykernel_launcher.py [-h] (--name <name> | --address <address>)
                             [--macos-use-bdaddr] [-d]
                             <notify uuid>
ipykernel_launcher.py: error: the following arguments are required: <notify uuid>


SystemExit: 2

In [32]:
import argparse
import asyncio

from bleak import BleakScanner


async def main(args: argparse.Namespace):
    print("scanning for 5 seconds, please wait...")

    devices = await BleakScanner.discover(
        return_adv=True,
        service_uuids=args.services,
        cb=dict(use_bdaddr=args.macos_use_bdaddr),
    )

    for d, a in devices.values():
        print()
        print(d)
        print("-" * len(str(d)))
        print(a)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()

    parser.add_argument(
        "--services",
        metavar="<uuid>",
        nargs="*",
        help="UUIDs of one or more services to filter for",
    )

    parser.add_argument(
        "--macos-use-bdaddr",
        action="store_true",
        help="when true use Bluetooth address instead of UUID on macOS",
    )

    args = parser.parse_args()

    await (main(args))

usage: ipykernel_launcher.py [-h] [--services [<uuid> ...]]
                             [--macos-use-bdaddr]
ipykernel_launcher.py: error: unrecognized arguments: --f=c:\Users\jarno\AppData\Roaming\jupyter\runtime\kernel-v3191a8289fad420feab075e915b839f3971fa594f.json


SystemExit: 2

c:\Users\jarno\Desktop\Random_Code\Random_Code\Python\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
